### STEP: 1

Import necessary libraries for **Predicting Future Stock Prices**

In [75]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import requests

import warnings
warnings.filterwarnings('ignore')

# Libraries to Extract Stock Price Historical Data 
# from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar

# Library for Technical Analysis
import pandas_ta as ta

# Libraries for ML Analysis using Linear Models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge

scaler = StandardScaler()
today = datetime.today().strftime('%Y-%m-%d')

### STEP: 2

Defining various functions used in the code for 
* extracting data, 
* adding technical indicators as features,
* predicting **Close** price and **VWAP** volume-weighted average price,
* using linear models for predicting with best model output.

Best model is selected based on minimum MAPE & RMSE

In [76]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
        
def extract_stock_data(stock_name, start_date):
    row_index = ticker_df.query(f"Security == '{stock_name}'").index[0]
    stock_ticker = ticker_list[row_index]
    OHLCV_df = yf.download(stock_ticker,start_date,today)
    OHLCV_df.index = pd.to_datetime(OHLCV_df.index)
    print(f"{stock_ticker}:{stock_name}")
    #display(OHLCV_df.head(3))
    #display(OHLCV_df.tail(3))
    return(OHLCV_df)

# Feature Engineering _ additionals features for ML Analysis

def technical_indicators():
    OHLCV_df = extract_stock_data(stock_name, start_date)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # Daily Returns
    OHLCV_df['daily_returns'] = OHLCV_df['Close'].pct_change()
    # 21Day Volatility
    OHLCV_df['volatility_21'] = OHLCV_df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    OHLCV_df['sma_21'] = OHLCV_df.ta.sma(length=21)
    OHLCV_df['sma_50'] = OHLCV_df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    OHLCV_df['ema_21'] = OHLCV_df.ta.ema(length=21)
    OHLCV_df['ema_50'] = OHLCV_df.ta.ema(length=50)
    # RSI Momemtum Indicator 14Days
    OHLCV_df['rsi_14'] = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    OHLCV_df[['ema_12', 'ema_26', 'MACD_signal']] = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    OHLCV_df[['BB_lower','BB_middle','BB_upper','BBP','BB%B']]= OHLCV_df.ta.bbands()
    # On-Balance Volume in Millions
    OHLCV_df['OBV_in_million'] = OHLCV_df.ta.obv()/1000000
    # Average Directional Index 14Days
    OHLCV_df[['adx_14','dmp_14','dmn_14']] = OHLCV_df.ta.adx()
    # Dropping trivial Columns
    OHLCV_append_df = OHLCV_df.drop(columns = ['Open','High','Low','Adj Close',
                                               'typical_price','vw_typical_price',
                                               'BBP','dmp_14','dmn_14']).dropna()
    #display(OHLCV_append_df.tail())
    
    return(OHLCV_append_df)


In [77]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    list_of_models = [Ridge(),Lasso(),MultiTaskElasticNet(),
                  LassoLars(),BayesianRidge()]
    target_predictions_close_df = target_test_close.copy()
    for model in list_of_models:
        model_instance_close = model
        model_instance_close.fit(predictors_train_scaled,target_train_close)
        model_prediction_close = model_instance_close.predict(predictors_test_scaled)
        target_predictions_close_df[f'{model}_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    for model in list_of_models:
        model_instance_vwap = model
        model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
        model_prediction_vwap = model_instance_vwap.predict(predictors_test_scaled)
        target_predictions_vwap_df[f'{model}_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    display(target_predictions_df.tail())
    display(target_predictions_close_df.hvplot(frame_width=500,frame_height=300))
    return(target_predictions_close_df,target_predictions_vwap_df,target_predictions_df)



In [78]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def linear_model_predictions():
    display(extract_stock_data(stock_name, start_date).head(3))
    display(extract_stock_data(stock_name, start_date).tail(3))
    display(technical_indicators().tail())
    today = datetime.today().strftime('%Y-%m-%d')
    target_close_predictions_df,target_vwap_predictions_df,target_predictions_df = predict_close_vwap()
    column_range_close = list(target_close_predictions_df.columns)
    column_range_vwap = list(target_vwap_predictions_df.columns)
    dict_close = {}
    dict_vwap = {}
    for i in column_range_close[1:]:
        rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])**2))
        mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'Close']))*100
        dict_close[f'{i}'] = [rmse_close,mape_close]
    for i in column_range_vwap[1:]:  
        rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])**2))
        mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'vwap_annual']))*100
        dict_vwap[f'{i}'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    error_measures_close = error_measures_close.sort_values(by=['MAPE_close'])
    error_measures_vwap = error_measures_vwap.sort_values(by=['MAPE_vwap'])

    display(error_measures_close)
    display(error_measures_close.hvplot.bar(rot=90,frame_width=500,frame_height=300))
    
    #Choosing model with lowest MAPE value
    best_estimate_close = error_measures_close.index[0]
    best_estimate_vwap = error_measures_vwap.index[0]

    predictions_output = target_predictions_df[['Close','vwap_annual',best_estimate_close, best_estimate_vwap]]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output[best_estimate_close])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output[best_estimate_vwap])

    predictions = predictions_output[['Close','vwap_annual']]
    best_model = best_estimate_close.removesuffix('_Close')
    comparison_plot = predictions_output[['Close',best_estimate_close]].loc['2024':today].hvplot(frame_width=500,frame_height=300) 
    
    return (comparison_plot,best_model,predictions)


In [80]:
# Scrapping for List of stocks in S&P500 for Linear Model ML Prediction Analysis
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
  
wiki_data = pd.read_html(url)

ticker_df = wiki_data[0]
display(ticker_df.head(2))
display(ticker_df.tail(2))

# Pick Stock from the list for analysis

ticker_list= ticker_df['Symbol'].to_list()
stock_list = ticker_df['Security'].to_list()
print(stock_list)
stock_name = input('Enter Stock_Name from the above displayed list for Analysis:')
start_date = input('Enter Start Date for Historical Data in yyyy-mm-dd:')


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
502,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'Adobe Inc.', 'Advanced Micro Devices', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Air Products and Chemicals', 'Airbnb', 'Akamai', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Archer-Daniels-Midland', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'Automated Data Processing', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corporation', 'Bank

Enter Stock_Name from the above displayed list for Analysis: Tapestry, Inc.
Enter Start Date for Historical Data in yyyy-mm-dd: 2012-01-01


In [82]:
# display predictions

model_plot,model,stock_predictions = linear_model_predictions()

display(stock_predictions.tail())

display(model_plot)

print(f"Model with lowest MAPE: {model}")

[*********************100%%**********************]  1 of 1 completed

TPR:Tapestry, Inc.


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,61.919998,61.919998,59.950001,60.040001,41.743847,2666000
2012-01-04,59.970001,61.980000,59.799999,61.590000,42.821503,2773100
2012-01-05,61.139999,62.009998,60.459999,61.860001,43.009235,2593700


[*********************100%%**********************]  1 of 1 completed

TPR:Tapestry, Inc.


...

,RMSE_close,MAPE_close
BayesianRidge()_Close,0.871955,2.025924
Ridge()_Close,0.882564,2.059867
LassoLars()_Close,1.306781,3.828972
Lasso()_Close,1.306781,3.828972
MultiTaskElasticNet()_Close,1.967089,5.876466


:Bars   [index,Variable]   (value)

,Close,vwap_annual
Date,,
2024-03-20,47.400002,42.692682
2024-03-21,48.090000,42.772883
2024-03-22,46.950001,42.822185
2024-03-25,45.549999,42.870382
2024-03-26,45.669727,42.924140


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

Model with lowest MAPE: BayesianRidge()


## Predicting Stock Price for future dates

In [83]:
# Feature Engineering _ for future prediction using stock_predictions as input
# OBV & ADX dropped to limit predicted features

def feature_engineering(df):
    # Daily Returns
    df['daily_returns'] = df['Close'].pct_change()
    # 21Day Volatility
    df['volatility_21'] = df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    df['sma_21'] = df['Close'].rolling(window=21).mean()
    df['sma_50'] = df['Close'].rolling(window=50).mean()
    # Exponential Moving Avg 21Days & 50Days
    df['ema_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['ema_50'] = df['Close'].ewm(span=50, adjust=False).mean()
     # RSI Momemtum Indicator 14Days
    df['rsi_14'] = df.ta.rsi()
    # MACD Momentum Indicator
    df[['ema_12', 'ema_26', 'MACD_signal']] = df.ta.macd()
    # Bolinger Bands %B Indicator
    df[['BB_lower','BB_middle','BB_upper','BBP','BB%B']]= df.ta.bbands()
    # Dropping trivial Columns
    append_df = df.drop(columns = ['BBP']).dropna()
    trim_df = df[['Close','vwap_annual']]
    return(append_df)


In [84]:
# Creating instance of best fit model _ BayesianRidge()
# By fitting the historical data

def best_fit_model_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns=['Volume','OBV_in_million','adx_14'])
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model = BayesianRidge()
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model}_Close'] = model_prediction_close[-1]

    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model}_vwap_annual'] = model_prediction_vwap[-1]

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model}_vwap_annual'])

    predictions = target_future_df[['Close','vwap_annual']]
    return (predictions)


In [85]:
#Predicting Day1
future_stock_price = best_fit_model_future_prediction(stock_predictions)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,14)):
    future_stock_price = best_fit_model_future_prediction(future_stock_price)
    
#Display Future Stock Price Predictions
display(future_stock_price.loc[today:,:])

future_plot = future_stock_price.loc[today:,'Close'].hvplot(frame_width=500,frame_height=300)

display(future_plot)
display(model_plot*future_plot)

[*********************100%%**********************]  1 of 1 completed


TPR:Tapestry, Inc.


,Close,BayesianRidge()_Close,vwap_annual,BayesianRidge()_vwap_annual
Date,,,,
2024-03-26,NaN,45.799256,NaN,42.989401


[*********************100%%**********************]  1 of 1 completed


TPR:Tapestry, Inc.


...

TPR:Tapestry, Inc.


,Close,BayesianRidge()_Close,vwap_annual,BayesianRidge()_vwap_annual
Date,,,,
2024-04-12,NaN,45.795743,NaN,43.822458


,Close,vwap_annual
Date,,
2024-03-26,45.669727,42.924140
2024-03-27,45.799256,42.989401
2024-03-28,45.891684,43.065349
2024-03-29,45.922252,43.118934
2024-04-01,45.865717,43.144380
2024-04-02,45.828343,43.191407
2024-04-03,45.837013,43.262677
2024-04-04,45.850715,43.341406
2024-04-05,45.833736,43.409287


:Curve   [Date]   (Close)

:Overlay
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Curve.Close :Curve   [Date]   (Close)

## Predicting Using Regressor Models

In [86]:
# Single Target Regressor Models
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

svr_rbf = SVR(kernel='rbf',C=100,gamma='auto')
# svr_poly = SVR(kernel='poly',C=100,degree=2, gamma='auto')

knn_uniform = KNeighborsRegressor(n_neighbors=2,weights='uniform')
knn_distance = KNeighborsRegressor(n_neighbors=5,weights='distance')

dtree = DecisionTreeRegressor(max_depth=10)

ens = MultiOutputRegressor(GradientBoostingRegressor(random_state=0))

# mlp = MLPRegressor(random_state=1, max_iter=100)

In [87]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_regressor_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    list_of_regressors = [svr_rbf,knn_uniform,knn_distance,dtree,ens]
    target_predictions_close_df = target_test_close.copy()
    for model in list_of_regressors:
        model_instance_close = model
        model_instance_close.fit(predictors_train_scaled,target_train_close)
        model_prediction_close = model_instance_close.predict(predictors_test_scaled)
        target_predictions_close_df[f'{model}_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    for model in list_of_regressors:
        model_instance_vwap = model
        model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
        model_prediction_vwap = model_instance_vwap.predict(predictors_test_scaled)
        target_predictions_vwap_df[f'{model}_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    display(target_predictions_df.tail())
    display(target_predictions_close_df.hvplot(frame_width=500,frame_height=300))
    return(target_predictions_close_df,target_predictions_vwap_df,target_predictions_df)


In [88]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def regressor_model_predictions():
    display(extract_stock_data(stock_name, start_date).head(3))
    display(extract_stock_data(stock_name, start_date).tail(3))
    display(technical_indicators().tail())
    today = datetime.today().strftime('%Y-%m-%d')
    target_close_predictions_df,target_vwap_predictions_df,target_predictions_df = predict_regressor_close_vwap()
    column_range_close = list(target_close_predictions_df.columns)
    column_range_vwap = list(target_vwap_predictions_df.columns)
    dict_close = {}
    dict_vwap = {}
    for i in column_range_close[1:]:
        rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])**2))
        mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'Close']))*100
        dict_close[f'{i}'] = [rmse_close,mape_close]
    for i in column_range_vwap[1:]:  
        rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])**2))
        mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'vwap_annual']))*100
        dict_vwap[f'{i}'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    error_measures_close = error_measures_close.sort_values(by=['MAPE_close'])
    error_measures_vwap = error_measures_vwap.sort_values(by=['MAPE_vwap'])

    display(error_measures_close)
    display(error_measures_close.hvplot.bar(rot=90,frame_width=700,frame_height=300))

    best_estimate_close = error_measures_close.index[0]
    best_estimate_vwap = error_measures_vwap.index[0]

    predictions_output = target_predictions_df[['Close','vwap_annual',best_estimate_close, best_estimate_vwap]]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output[best_estimate_close])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output[best_estimate_vwap])

    predictions = predictions_output[['Close','vwap_annual']]
    best_model = best_estimate_close.removesuffix('_Close')
    comparison_plot = predictions_output[['Close',best_estimate_close]].loc['2024':today].hvplot(frame_width=300,frame_height=300)
    
    return (comparison_plot,best_model,predictions)



In [89]:
regressor_plot,regressor,stock_predictions_regr = regressor_model_predictions()

display(stock_predictions_regr.tail())
print(f'Best fit Regressor Model: {regressor}')
regressor_plot


[*********************100%%**********************]  1 of 1 completed

TPR:Tapestry, Inc.


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-01-03,61.919998,61.919998,59.950001,60.040001,41.743847,2666000
2012-01-04,59.970001,61.980000,59.799999,61.590000,42.821503,2773100
2012-01-05,61.139999,62.009998,60.459999,61.860001,43.009235,2593700


[*********************100%%**********************]  1 of 1 completed

TPR:Tapestry, Inc.


...

,RMSE_close,MAPE_close
MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=0))_Close,1.213167,3.068930
DecisionTreeRegressor(max_depth=10)_Close,1.897450,4.642196
KNeighborsRegressor(weights='distance')_Close,2.965972,7.671059
KNeighborsRegressor(n_neighbors=2)_Close,3.205041,8.304689
"SVR(C=100, gamma='auto')_Close",3.867285,8.491395


:Bars   [index,Variable]   (value)

,Close,vwap_annual
Date,,
2024-03-20,47.400002,42.692682
2024-03-21,48.090000,42.772883
2024-03-22,46.950001,42.822185
2024-03-25,45.549999,42.870382
2024-03-26,45.670390,42.248324


Best fit Regressor Model: MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=0))


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [90]:
# Creating instance of best fit regressor model
# By fitting the historical data

def best_fit_regr_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns=['Volume','OBV_in_million','adx_14'])
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model_close = ens
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model_close
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model_close}_Close'] = model_prediction_close

    best_model_vwap = ens
    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model_vwap
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model_vwap}_vwap_annual'] = model_prediction_vwap

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model_close}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model_vwap}_vwap_annual'])

    predictions = target_future_df[['Close','vwap_annual']]
    
    return (predictions)


In [91]:
#Predicting Day1
future_stock_price_regr = best_fit_regr_future_prediction(stock_predictions_regr)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,14)):
    future_stock_price_regr = best_fit_regr_future_prediction(future_stock_price_regr)

#Display Regressor Model Predictions
display(future_stock_price_regr.loc[today:,:])

future_plot_regr = future_stock_price_regr.loc[today:,'Close'].hvplot(frame_width=300,frame_height=300)

display(future_plot_regr)
display(regressor_plot*future_plot_regr)

[*********************100%%**********************]  1 of 1 completed


TPR:Tapestry, Inc.


,Close,MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=0))_Close,vwap_annual,MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=0))_vwap_annual
Date,,,,
2024-03-26,NaN,45.882519,NaN,41.839717


[*********************100%%**********************]  1 of 1 completed


TPR:Tapestry, Inc.


...

TPR:Tapestry, Inc.


,Close,MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=0))_Close,vwap_annual,MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=0))_vwap_annual
Date,,,,
2024-04-12,NaN,45.795072,NaN,42.032609


,Close,vwap_annual
Date,,
2024-03-26,45.670390,42.248324
2024-03-27,45.882519,41.839717
2024-03-28,46.204653,41.837318
2024-03-29,46.168164,41.837318
2024-04-01,46.125966,41.842700
2024-04-02,46.077696,41.855124
2024-04-03,46.117207,41.849742
2024-04-04,46.164422,41.849742
2024-04-05,46.164422,41.859544


:Curve   [Date]   (Close)

:Overlay
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Curve.Close :Curve   [Date]   (Close)

In [92]:
regressor_plot*future_plot*future_plot_regr

:Overlay
   .NdOverlay.I    :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Curve.Close.I  :Curve   [Date]   (Close)
   .Curve.Close.II :Curve   [Date]   (Close)

In [93]:
linear_close = future_stock_price[['Close']].rename(columns={'Close':'Close_Linear'})
regr_close = future_stock_price_regr[['Close']].rename(columns={'Close':'Close_Regr'})

combined_df = pd.concat([linear_close,regr_close],axis=1).dropna()

display(combined_df.loc[today:])

combined_df.loc[today:].hvplot()


,Close_Linear,Close_Regr
Date,,
2024-03-26,45.669727,45.670390
2024-03-27,45.799256,45.882519
2024-03-28,45.891684,46.204653
2024-03-29,45.922252,46.168164
2024-04-01,45.865717,46.125966
2024-04-02,45.828343,46.077696
2024-04-03,45.837013,46.117207
2024-04-04,45.850715,46.164422
2024-04-05,45.833736,46.164422


:NdOverlay   [Variable]
   :Curve   [Date]   (value)